In [ ]:
import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow
tensorflow.config.list_physical_devices("GPU")

#### Design the Model

In [ ]:
def get_compiled_model():
   
    inputs = keras.Input(shape=(28,28,1))
    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform')(inputs)
    x = keras.layers.MaxPool2D(pool_size=(2,2))(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

#### Load the Dataset
When doing distributed training, the efficiency with which you load data can often become critical. It is recommended to use tf.data.Dataset objects to load data in a multi-device or distributed workflow.

In [ ]:
def get_dataset():
    batch_size = 64    # global batch size = batch_size_per_replica (depending on memory of each GPU) * # gpus
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a `tf.data.Dataset`.
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    numTrain = x_train.shape[0]
    numTest = x_test.shape[0]
    print("train size: {}, test size: {}".format(x_train.shape, x_test.shape))

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(numTrain, 28, 28, 1)
    x_test = x_test.reshape(numTest, 28, 28, 1)
    
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),   # Creates a `Dataset` whose elements are slices of the given tensors.
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )

#### Instantialte a Strategy

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

#### Open Scope Using the Strategy Object

In [ ]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

#### Fit the Model

In [ ]:
# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1)

#### Evaluate the Model

In [ ]:
# Test the model on all available devices.
model.evaluate(test_dataset)